# Use Studio notebook lifecycle configurations

This notebook implements a method for configuring dependencies within SageMaker Studio by using Studio Lifecycle Configuration whereby dependencies are installed each time a kernel starts-up.

For this notebook, please use the <b>Data Science</b> image and <b>Python 3</b> kernel.

<div class="alert alert-block alert-warning">
    This notebook requires the <b>Studio Execution Role</b> to have <b>sagemaker:UpdateDomain</b> and <b>sagemaker:UpdateUserProfile</b> permissions
</div>

## Configure environment

In [2]:
import os
import json
import random
import base64
import boto3
import botocore.exceptions
import sagemaker

client = boto3.client('sagemaker')

Create Studio Lifecyle Configuration

In [3]:
StudioLifecycleConfigName = 'InstallDependencies-%004x' % random.getrandbits(16)

StudioLifecycleConfigContent = """# This script installs a single pip package on a SageMaker Studio Kernel Application
#!/bin/bash
set -eux
# PARAMETERS
PACKAGE=pyarrow
pip install --upgrade $PACKAGE"""

message_bytes = StudioLifecycleConfigContent.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)

studio_lifecycle_config = client.create_studio_lifecycle_config(
    StudioLifecycleConfigName=StudioLifecycleConfigName,
    StudioLifecycleConfigContent=base64_bytes.decode('ascii'),
    StudioLifecycleConfigAppType='KernelGateway',
)

Fetch the SageMaker Studio domain ID

In [4]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        domain_id = json.loads(f.read()).get('DomainId')

There two possible options for the next step, option 1 is adding the lifecycle configuration to the SageMaker domain so that all profiles within that domain inherit it and option 2 is to add the lifecycle configuration to a user profile so that it is scoped to that specific profile.

Option 1: Add Studio Lifecyle Configuration to the SageMaker domain

In [5]:
domain = client.describe_domain(
    DomainId=domain_id
)
lifecycle_config_arns = domain['DefaultUserSettings']['KernelGatewayAppSettings']['LifecycleConfigArns']
lifecycle_config_arns.append(studio_lifecycle_config['StudioLifecycleConfigArn'])

try:
    client.update_domain(
        DomainId=domain_id,
        DefaultUserSettings={
            'KernelGatewayAppSettings': {
                'LifecycleConfigArns': lifecycle_config_arns,
            }
        }
    )
except botocore.exceptions.ClientError as e:
    role = sagemaker.get_execution_role()
    print(f"SageMaker Studio execution role ({role}) does not have permission to perform UpdateDomain operation. Please add 'sagemaker:UpdateDomain' permission to the role.")
    raise

Option 2: Add Studio Lifecyle Configuration to a specific user profile.

In [6]:
user_profile_name = 'alex'


user_profile = client.describe_user_profile(
    DomainId=domain_id,
    UserProfileName=user_profile_name
)

try:
    lifecycle_config_arns = user_profile['UserSettings']['KernelGatewayAppSettings']['LifecycleConfigArns']
except KeyError as e:
    lifecycle_config_arns = []
lifecycle_config_arns.append(studio_lifecycle_config['StudioLifecycleConfigArn'])

try:
    client.update_user_profile(
        DomainId=domain_id,
        UserProfileName=user_profile_name,
        UserSettings={
            'KernelGatewayAppSettings': {
                'LifecycleConfigArns': lifecycle_config_arns,
            }
        }
    )
except botocore.exceptions.ClientError as e:
    role = sagemaker.get_execution_role()
    print(f"SageMaker Studio execution role ({role}) does not have permission to perform UpdateUserProfile operation. Please add 'sagemaker:UpdateUserProfile' permission to the role.")
    raise

Now that all the configuration is done, it is time to test the script within Studio. To do this, launch the Studio and on the Launcher tab, locate the Notebooks and compute resources section and click on the Change environment to select the lifecycle configuration you created.

![](../img/studio-create-notebook.png)

On the Change environment popup, change the Start-up script option to be the lifecycle configuration created in previous steps.

![](../img/change-environment-2.png)

Back in the Launcher window, click on the Create notebook option.
You can also set the Lifecycle configuration to be run by default in the Lifecycle configurations for personal Studio apps section of the Domain page.
Within the new notebook, the dependencies installed in the start-up script will be available.

![](../img/lifecycle-config-working.png)